In [1]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40

log:
    root: '?'
    save_preds: '?' 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: 1 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0     
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: '?'

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "ISLES_3D_TempSweep"

experiment: 
    inf_kwargs:
        threshold: 0.5 
        temperature: 
            - (0.01, ..., 1, 0.05)
            - (1.01, ..., 2, 0.05)
            - (2.01, ..., 3, 0.05)
            - (3.01, ..., 4, 0.05)

base_model: 
    - "/storage/vbutoi/scratch/ESE/training/09_25_24_ISLES_3D_Dice_HeavyAug/20240925_234556-HGRQ-56cd3bf7df7a1b7d2a453b296d64407d"
    # - "/storage/vbutoi/scratch/ESE/calibration/09_29_24_ISLES_3D_LTS_Bs4_lr1e-4_Huge_v2/20240929_113434-01QO-0bd7e7617309fdcacdf0080be7c83804"

model:
    checkpoint: 'min-val-abs_area_estimation_error'

inference_data:
    _class: "ese.datasets.ISLES"
    version: 1.0
    split: ("cal", "val")

log:
    save_preds: True 

dataloader:
    batch_size: 1

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

{'base_model': ['/storage/vbutoi/scratch/ESE/training/09_25_24_ISLES_3D_Dice_HeavyAug/20240925_234556-HGRQ-56cd3bf7df7a1b7d2a453b296d64407d'],
 'dataloader.batch_size': 1,
 'experiment.inf_kwargs.temperature': ['(0.01, ..., 1, 0.05)',
                                       '(1.01, ..., 2, 0.05)',
                                       '(2.01, ..., 3, 0.05)',
                                       '(3.01, ..., 4, 0.05)'],
 'experiment.inf_kwargs.threshold': 0.5,
 'inference_data.split': '("cal", "val")',
 'inference_data.version': 1.0,
 'log.save_preds': True,
 'model.checkpoint': 'min-val-abs_area_estimation_error'}
{'base_model': ['/storage/vbutoi/scratch/ESE/training/09_25_24_ISLES_3D_Dice_HeavyAug/20240925_234556-HGRQ-56cd3bf7df7a1b7d2a453b296d64407d'],
 'dataloader.batch_size': 1,
 'experiment.inf_kwargs.temperature': ['(0.01, ..., 1, 0.05)',
                                       '(1.01, ..., 2, 0.05)',
                                       '(2.01, ..., 3, 0.05)',
               

ValueError: Stop here.

In [ ]:
len(inf_cfgs)

In [ ]:
print(inf_cfgs[0].to_dict())

## Running Jobs

In [ ]:
from ese.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='0',
) 

In [ ]:
# from ese.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     group="inference",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
# )